Воольные рассуждения на тему детекции грида шашек. Возможно что то отсюда еще
будет полезно

In [2]:
import numpy as np 
import cv2
import os
from matplotlib import pyplot as plt
def imshow(image):
    cv2.imshow('image', image)
    cv2.waitKey(0) 
    cv2.destroyWindow('image')

In [ ]:
#%matplotlib inline

In [ ]:
curDir = os.getcwd()
print(os.path.join(curDir,"test.jpg"))
image = cv2.imread(os.path.join(curDir,"test.jpg"))
#for some reason ipynb can't work with cv2, so we have to do this teribleness

#imshow(image)

In [ ]:
grayed = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
#imshow(grayed)

In [ ]:
blur = cv2.GaussianBlur(grayed, (5,5), 0)
#imshow(blur)

In [ ]:
thresh = cv2.adaptiveThreshold(blur, 255, 1, 1, 11, 2)
#imshow(thresh)

In [ ]:
contours, _ = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

max_area = 0
c = 0
for i in contours:
        area = cv2.contourArea(i)
        if area > 1000:
                if area > max_area:
                    max_area = area
                    best_cnt = i
                    image = cv2.drawContours(image, contours, c, (0, 255, 0), 3)
        c+=1

mask = np.zeros((grayed.shape),np.uint8)
cv2.drawContours(mask,[best_cnt],0,255,-1)
cv2.drawContours(mask,[best_cnt],0,0,2)
imshow(mask)

In [ ]:
out = np.zeros_like(image)
out[mask == 255] = image[mask == 255]
imshow(out)

In [ ]:
blur = cv2.GaussianBlur(out, (5,5), 0)
thresh = cv2.adaptiveThreshold(blur, 255, 1, 1, 11, 2)
contours, _ = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
c = 0
for i in contours:
        area = cv2.contourArea(i)
        if area > 1000/2:
            cv2.drawContours(image, contours, c, (0, 255, 0), 3)
        c+=1
imshow(image)

method 2

In [ ]:
curDir = os.getcwd()
imDir = os.path.join(curDir,"test.jpg")
image= cv2.imread(imDir)

In [ ]:
gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
gray = np.float32(gray)
dst = cv2.cornerHarris(gray,2,3,0.04)
#result is dilated for marking the corners, not important
dst = cv2.dilate(dst,None)
# Threshold for an optimal value, it may vary depending on the image.
image[dst>0.01*dst.max()]=[0,0,255]
cv2.imshow('dst',image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
image= cv2.imread('Boxes.png')
gray= cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
gray= np.float32(gray)

harris_corners= cv2.cornerHarris(gray, 3, 3, 0.05)

kernel= np.ones((7,7), np.uint8)

harris_corners= cv2.dilate(harris_corners, kernel, iterations= 2)


image[harris_corners > 0.025 * harris_corners.max()]= [255,127,127]

cv2.imshow('Harris Corners', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

method 3 

In [ ]:
curDir = os.getcwd()
imDir = os.path.join(curDir,"test.jpg")
image = cv2.imread(imDir)

In [ ]:
import cv2

gray= cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)


corners= cv2.goodFeaturesToTrack(gray, 100, 0.01, 50)

for corner in corners:
    x,y= corner[0]
    x= int(x)
    y= int(y)
    cv2.rectangle(image, (x-10,y-10),(x+10,y+10),(255,0,0),-1)



cv2.imshow("goodFeaturesToTrack Corner Detection", image)
cv2.waitKey()
cv2.destroyAllWindows()



method 4(back to drawing lines)

In [2]:
import scipy.spatial as spatial
import scipy.cluster as cluster
from collections import defaultdict

# Hierarchical cluster (by euclidean distance) intersection points
def clusterPoints(points):
    dists = spatial.distance.pdist(points)
    single_linkage = cluster.hierarchy.single(dists)
    flat_clusters = cluster.hierarchy.fcluster(single_linkage, 10,'distance')
    cluster_dict = defaultdict(list)
    for i in range(len(flat_clusters)):
        cluster_dict[flat_clusters[i]].append(points[i])
    cluster_values = cluster_dict.values()
    clusters = map(lambda arr: (np.mean(np.array(arr)[:, 0]), np.mean(np.array(arr)[:, 1])), cluster_values)
    return np.array(sorted(list(clusters), key=lambda k: [k[1], k[0]]))

def augmentPoints(points):
    points_shape = list(np.shape(points))
    augmented_points = []
    for row in range(int(points_shape[0] / 11)):
        start = row * 11
        end = (row * 11) + 10
        rw_points = points[start:end + 1]
        rw_y = []
        rw_x = []
        for point in rw_points:
            x, y = point
            rw_y.append(y)
            rw_x.append(x)
        y_mean = np.mean(np.array(rw_y))
        for i in range(len(rw_x)):
            point = (rw_x[i], y_mean)
            augmented_points.append(np.int32(point))
    augmented_points = sorted(augmented_points, key=lambda k: [k[1], k[0]])
    return np.array(augmented_points)


In [3]:
import math

def getIntersections(hLines,vLines):
    points = []
    for vline in vLines:
        vline = vline[0]
        for hline in hLines:
            hline = hline[0]
            s = np.vstack([[vline[0],vline[1]],[vline[2],vline[3]],[hline[0],hline[1]],[hline[2],hline[3]]])        # s for stacked
            h = np.hstack((s, np.ones((4, 1)))) # h for homogeneous
            l1 = np.cross(h[0], h[1])           # get first line
            l2 = np.cross(h[2], h[3])           # get second line
            x, y, z = np.cross(l1, l2)          # point of intersection
            if z != 0:
                points.append([np.int32(x/z),np.int32(y/z)])
    return np.array(points)                  

#def hvSplit(lines):
#    hLines= []
#    vLines= []
#    for line in lines:
#        # line = [x1,y1,x2,y2]
#        x1,y1,x2,y2 = line[0]
#        A = y1-y2
#        B = x2-x1
#        if x2==x1 or np.arctan(-1*(A/B)) <:
#            hLines.append(line)
#        elif abs(line[0][1]-line[0][3]) < 50:
#            vLines.append(line)
#    return np.array(hLines),np.array(vLines)

def hvSplit(lines):
    hLines= []
    vLines= []
    const = 100
    for line in lines:
        if abs(line[0][0]-line[0][2])< const:
            hLines.append(line)
        elif abs(line[0][1]-line[0][3]) < const:
            vLines.append(line)
    return np.array(hLines),np.array(vLines)

def maskImage(img):
    grayed = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(grayed, (5,5), 0)
    thresh = cv2.adaptiveThreshold(blur, 255, 1, 1, 11, 2)
    contours, _ = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    max_area = 0
    c = 0
    for i in contours:
            area = cv2.contourArea(i)
            if area > 1000:
                    if area > max_area:
                        max_area = area
                        best_cnt = i
                        #cv2.drawContours(img, contours, c, (0, 255, 0), 3)
            c+=1
    mask = np.zeros((grayed.shape),np.uint8)
    cv2.drawContours(mask,[best_cnt],0,255,-1)
    cv2.drawContours(mask,[best_cnt],0,0,2)
    out = np.zeros_like(img)
    out[mask == 255] = img[mask == 255]
    return out

def findPointsPerline(points):
    res = []
    # TODO figure out smarter way to determine the epsilon
    epsilon = 10 
    lineStart = 0
    print(points.shape)
    a , _ = points.shape 
    index = 0 
    while index < a - 1:
        j = index
        lineStart = points[j]
        tmp = []
        while j < a - 1 and abs(lineStart[1]-points[j][1]) < epsilon :
            tmp.append(list(points[j]))
            j += 1
        res.append(sorted(tmp))
        index = j 
    return np.array(res) 

def distance(point1, point2):
    return math.sqrt((point1[0]-point2[0])**2+ (point1[1]-point2[1])**2)

#как же мне стыдно за это 
def correctPoints(points):
    points = points.tolist()
    epsilon = 10
    # correct xs
    for i in range(len(points)-1):
        if len(points[i]) < 1:
            points.pop(i)
    for i in range(len(points)):
        line = points[i]
        diffs = []
        for j in range(len(line)-1):
            diffs.append(distance(line[j],line[j+1]))
        diffs = np.array(diffs)
        mean = np.mean(diffs)
        newLine = []
        for j in range(len(line)):
            if j == len(line)-1 :
                if distance(line[j],line[j-1])  >= mean - epsilon:
                    newLine.append(line[j])
                continue
            if j == 0 :
                if distance(line[j],line[j+1])  >= mean - epsilon:
                   newLine.append(line[j])
                continue
            if distance(line[j],line[j+1])  >= mean - epsilon or  distance(line[j],line[j-1])  >= mean - epsilon:
                newLine.append(line[j])
        
        points[i] = newLine
    
    # we dont need lines with 1 or 2 points
    counter = 0 
    for i in range(len(points)):
        counter+=len(points[i])
    
    counter = int(counter/len(points))
    tmp = []
    for point in points:
        if len(point) >= counter/2:
            tmp.append(point)
    points = tmp
    
    #correct ys
    diffs = []
    for i in range(len(points)-2):
        if len(points[i]) <= 1 or len(points[i+1]) <= 1: 
            continue
        diffs.append(distance(points[i][0],points[i+1][0]))
    mean = np.mean(np.array(diffs))
    res = []
    
    epsilon = mean/3
    for i in range(len(points)-1):
        
        if i == len(line)-1 :
            if distance(points[i][0], points[i-1][0])  >= mean - epsilon:
                res.append(points[i])
            continue
        if i == 0 :
            if distance(points[i][0], points[i+1][0])  >= mean - epsilon:
                res.append(points[i])
            continue
        if distance(points[i][0], points[i+1][0]) >= mean - epsilon or  distance(points[i][0], points[i-1][0])  >= mean - epsilon:
            res.append(points[i])
    return res
         

    

In [4]:
def placeCheckers(points, checkers):
    field = np.zeros(shape = (11,11))
    for ch in checkers:
        xmin,ymin,xmax,ymax,cl = ch
        mindist1 = mindist2 = 100000
        minI = minJ = 0 
        for i in range(len(points)-1):
            line1 = points[i]
            line2 = points[i+1]
            for j in range(min(len(line1),len(line2))-1):
                dist1 = math.sqrt((xmin - line1[j][0] )**2 +(ymin -line1[j][1])**2)
                dist2 = math.sqrt((xmax -line2[j+1][0] )**2+(ymax -line2[j+1][1])**2 )
                if dist1 < mindist1 and dist2 < mindist2:
                    mindist1 = dist1
                    mindist2 = dist2
                    minI = i 
                    minJ = j  
        field[minI,minJ] = cl + 1
    return field
                
                

In [5]:
curDir = os.getcwd()
imDir = os.path.join(curDir,"test2.jpg")
img1 = cv2.imread(imDir)
# Loads an image
#masked = maskImage(img)
#imshow(masked)
gray= cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
blur = cv2.GaussianBlur(gray, (5,5), 0)
edges = cv2.Canny(blur,50,150,apertureSize = 3)
minLineLength=100
lines = cv2.HoughLinesP(image=edges,rho=1,theta=np.pi/180, threshold=90,lines=np.array([]), minLineLength=minLineLength,maxLineGap=90)

#print(lines)
hLines, vLines = hvSplit(lines)
points = getIntersections(hLines, vLines)
points = clusterPoints(points)
# Final coordinates of the board
#points = augmentPoints(points)
#print(points)
#print(points.shape)

#print(lines[0])
#lines = np.reshape(lines, (-1, 2))
#print(lines[0:2])
#h_lines, v_lines = h_v_lines(lines)

#a,b,c = lines.shape 
#for i in range(a):
#    cv2.line(img1, (lines[i][0][0], lines[i][0][1]), (lines[i][0][2], lines[i][0][3]), (0, 0, 255), 3, cv2.LINE_AA)

#a,b,c = hLines.shape 
#for i in range(a):
#    cv2.line(img1, (hLines[i][0][0], hLines[i][0][1]), (hLines[i][0][2], hLines[i][0][3]), (0, 0, 255), 3, cv2.LINE_AA)
#a,b,c = vLines.shape 
#for i in range(a):
#    cv2.line(img1, (vLines[i][0][0], vLines[i][0][1]), (vLines[i][0][2], vLines[i][0][3]), (255, 0, 0), 3, cv2.LINE_AA)
#print(points)
heh = findPointsPerline(points)
heh = correctPoints(heh)
#a = len(points) 
#for i in range(10,15):
#    cv2.circle(img1, (np.int32(points[i][0]),np.int32(points[i][1])), radius=4, color=(0, 0, 255), thickness=-1)   
#print(heh[0])
#
for j in range(len(heh)):
    color = (np.random.randint(0,255),np.random.randint(0,255),np.random.randint(0,255))
    for i in range(len(heh[j])):
        cv2.circle(img1, (np.int32(heh[j][i][0]),np.int32(heh[j][i][1])), radius=4, color=color, thickness=-1)   

#cv2.imwrite("res.jpg",img)# 53 329 753 329]]

imshow(img1)

(108, 2)


C:\Users\vvpvo\AppData\Local\Temp\ipykernel_11332\3038262765.py:82: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(res)


Extracting Coordinates from Yolo

In [6]:
from yolov5 import detect
import torch
import pandas as pd 
curDir = os.getcwd()
imDir = os.path.join(curDir,"test2.jpg")
print(imDir)
img = cv2.imread(imDir)
print(curDir+ "\\models\\detector.pt")


model = torch.hub.load('D:\\comptech\\checkersDetector\\yolov5', "custom",path="D:\\comptech\\checkersDetector\\models\\detector.pt", device="cpu",source='local') # local
res = model(img)

df = res.pandas().xyxy[0] 
df
         

YOLOv5  v6.0-216-g856d4e5 torch 1.10.1+cu113 CPU



d:\comptech\checkersDetector\test2.jpg
d:\comptech\checkersDetector\models\detector.pt


Fusing layers... 
Model Summary: 213 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 
d:\comptech\venv\lib\site-packages\torch\autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


,xmin,ymin,xmax,ymax,confidence,class,name
0,460.453979,56.688042,495.939514,103.939964,0.818788,0,black
1,271.230072,148.966660,312.098236,202.322769,0.810896,0,black
2,323.133911,106.203690,361.342041,156.429306,0.810081,0,black
3,111.939110,339.558807,157.733078,402.147797,0.801922,1,white
4,478.857910,413.135986,535.831726,471.443115,0.801752,1,white
5,212.875153,490.561371,264.070099,549.897461,0.797405,1,white
6,135.870209,104.367371,172.634430,154.819977,0.787938,0,black
7,158.593216,416.371735,207.127731,471.289337,0.783374,1,white
8,96.585152,494.102966,152.535583,552.370544,0.781999,1,white
9,319.485931,332.399139,363.688934,393.414337,0.780182,1,white


In [7]:
df = df[df.confidence>0.5] # img1 predictions (pandas)
df = df.drop(columns =["confidence","name"])
checkers = df.to_numpy()

print(checkers.shape)
a,_, = checkers.shape
for i in range(a):
    if checkers[i][4] == 1 :
        color = (255,0,0)
    else:
        color = (0,0,255)
    cv2.rectangle(img1, (np.int32(checkers[i][0]),np.int32(checkers[i][1])),(np.int32(checkers[i][2]),np.int32(checkers[i][3])), color = color)
    cv2.circle(img1, (np.int32(checkers[i][0]),np.int32(checkers[i][1])),radius=4, color=color, thickness=-1)
imshow(img1) 
field = placeCheckers(heh,checkers)
field
       

(24, 5)


array([[          0,           1,           0,           1,           0,           1,           0,           1,           0,           0,           0],
       [          1,           0,           1,           0,           1,           0,           1,           0,           0,           0,           0],
       [          0,           1,           0,           1,           0,           1,           0,           1,           0,           0,           0],
       [          0,           0,           0,           0,           0,           0,           0,           0,           0,           0,           0],
       [          0,           0,           0,           0,           0,           0,           0,           0,           0,           0,           0],
       [          2,           0,           2,           0,           2,           0,           2,           0,           0,           0,           0],
       [          0,           2,           0,           2,           0,           2,   

haha

In [7]:
from GridDetector import CheckersDetector
import cv2
pathToYolo = 'D:\\comptech\\checkersDetector\\yolov5'
pathToModel = "D:\\comptech\\checkersDetector\\models\\detector.pt"
debugPath = "D:\\comptech\\checkersDetector\\debug"
detector = CheckersDetector(pathToYolo=pathToYolo,pathToModel=pathToModel,debug=True, debugOutputPath = debugPath)
curDir = os.getcwd()
imDir = os.path.join(curDir,"test1.jpg")
print(imDir)
img = cv2.imread(imDir)
imshow(img)
res = detector.getGameField(img,visualize=True)
imshow(res[1])
imshow(res[0])

YOLOv5  v6.0-216-g856d4e5 torch 1.10.1+cu113 CPU

Fusing layers... 
Model Summary: 213 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


d:\comptech\checkersDetector\test1.jpg
(138, 2)


d:\comptech\checkersDetector\GridDetector.py:107: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(res)
d:\comptech\venv\lib\site-packages\numpy\core\fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
d:\comptech\venv\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
d:\comptech\venv\lib\site-packages\torch\autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


In [20]:
import numpy as np 
rr = np.array([
    [0,2,0,2,0,2,0,2],
    [2,0,2,0,2,0,2,0],
    [0,2,0,2,0,2,0,2],
    [0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0],
    [0,1,0,1,0,1,0,1],
    [1,0,1,0,1,0,1,0],
    [1,0,1,0,1,0,1,0]
])

In [30]:
def visualize(img, grid,field):

    centerx = (np.int32(grid[1][0][0]) + np.int32(grid[0][1][0]))/2
    centery = (np.int32(grid[1][0][1]) + np.int32(grid[0][0][1]))/2
    color = img[np.int32(centerx),np.int32(centery)]
    colors = [(0,0,0),(255,255,255)] # 0 - black  1 - white 
    
    if color[0] > 100 or color[1] > 100 or color[2] > 100:
        switch = 1 
    else:
        switch = 0 
    res = np.zeros((640,640,3), np.uint8)
    
    for y in range(8):
        for x in range(8):
            cv2.rectangle(res,(x*80,y*80),((x+1)*80,(y+1)*80), colors[switch],-1 )
            switch = (switch + 1) %2
        switch = (switch + 1) %2
    
    for y in range(8):
        for x in range(8):
            if field[y][x] == 1:
                cv2.circle(res,(80*x+40,80*y+40),35,(105,105,105),-1)
            elif field[y][x] == 2:
                cv2.circle(res,(80*x+40,80*y+40),35,(255,255,255),-1)
                
    return res 
print(rr)
res = visualize(img1,heh,rr)
imshow(res)

[[0 2 0 2 0 2 0 2]
 [2 0 2 0 2 0 2 0]
 [0 2 0 2 0 2 0 2]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [1 0 1 0 1 0 1 0]
 [0 1 0 1 0 1 0 1]
 [1 0 1 0 1 0 1 0]]
